In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json

from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    print(ds.StudyDescription)
    if ds.Modality == 'DX' and ds.PatientPosition in ['AP', 'PA']:
        img = ds.pixel_array
        return img
    else:
        return None
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    idg = ImageDataGenerator()
    img = resize(img, IMG_SIZE)
    proc_img = idg.flow(
            img,
            y=None,
            batch_size=32,
            shuffle=True,
            sample_weight=None,
            seed=None,
            save_to_dir=None,
            save_prefix="",
            save_format="png",
            subset=None,)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    jsonfile = open(model_path, 'r')
    model_json = jsonfile.read()
    jsonfile.close()

    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(weight_path)
    print("Loaded model from disk")

    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction_prob = model.predict(img, verbose = True)
    prediction = ['Pneumoinia' if i[0] > thresh else 'NOT Pneumoinia' for i in prediction_prob]
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' # path to saved model
weight_path = 'xray_class_my_model.best.hdf5' # path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 0 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.4 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Loaded model from disk
Load file test1.dcm ...
No Finding
1/1 [==============================] - 1s 1s/step
['Pneumoinia']
Load file test2.dcm ...
Cardiomegaly
1/1 [==============================] - 1s 1s/step
['Pneumoinia']
Load file test3.dcm ...
Effusion
1/1 [==============================] - 1s 1s/step
['Pneumoinia']
Load file test4.dcm ...
No Finding
1/1 [==============================] - 1s 1s/step
['Pneumoinia']
Load file test5.dcm ...
No Finding
Load file test6.dcm ...
No Finding
